# Data Preparation 2

Due to the poor results we got in the first round of modelling, we decided to make some adjustments to our dataset by conducting feature engineering. The 3 feature engineering steps we will take are:
1. Aggregating by region rather than planning area
2. Decomposing Wind Speeds into 2 Components
3. Calculating the Change in Weather Variables.

Import the libraries and read the raw data.

In [1]:
import pandas as pd
import numpy as np
import math
import geopandas as gpd
import fiona
import xmltodict
from shapely.geometry import Point
from IPython.display import IFrame

In [2]:
temperature_raw=pd.read_csv("temperature_raw_updated.csv")
humidity_raw=pd.read_csv("humidity_raw_updated.csv")
winddirection_raw=pd.read_csv("winddirection_raw_updated.csv")
windspeed_raw=pd.read_csv("windspeed_raw_updated.csv")
rainfall_raw=pd.read_csv("rainfall_raw_updated.csv")

### Adjustment 1: Aggregating by region rather than planning area

With 17 unique planning areas that the data could be from, we suspect that this variable was too granular resulting in a highly commplex dataset when Onehotencoded and thus a poor result. Hence we change the granularity to the 5 regions of Singapore. The steps taken are similar to getting the planning area but with a different <code>.kml</code> file.

In [3]:
map_url='https://data.gov.sg/dataset/master-plan-2014-region-boundary-web/resource/0bbbeedc-f7e1-46c6-bfe1-73f42220dea9/view/fd1e61f9-4ed2-4c15-8c8f-c5abd4aa9772'
IFrame(map_url, width=700, height=350)

In [4]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
df = gpd.read_file("region.kml", driver='KML')
df.head()

,Name,Description,geometry
0,CENTRAL REGION,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((103.84875 1.36303 0.00000, 1..."
1,EAST REGION,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((103.95322 1.38202 0.00000, 1..."
2,NORTH REGION,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((103.77664 1.45145 0.00000, 1..."
3,NORTH-EAST REGION,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((103.89715 1.41502 0.00000, 1..."
4,WEST REGION,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((103.69729 1.30754 0.00000, 1..."


In [18]:
df["Name"]=df["Name"].apply(lambda x: x.lower().split()[0])
df

,Name,Description,geometry
0,central,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((103.84875 1.36303 0.00000, 1..."
1,east,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((103.95322 1.38202 0.00000, 1..."
2,north,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((103.77664 1.45145 0.00000, 1..."
3,north-east,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((103.89715 1.41502 0.00000, 1..."
4,west,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","MULTIPOLYGON Z (((103.69729 1.30754 0.00000, 1..."


In [20]:
def get_region(latitude,longitude):
    p=Point(longitude,latitude)
    for i in range(len(df)):
        if p.within(df["geometry"][i]):
            return df["Name"][i]

In [22]:
temperature_raw["region"]=temperature_raw.apply(lambda x:get_region(x.latitude,x.longitude),axis=1)
humidity_raw["region"]=humidity_raw.apply(lambda x:get_region(x.latitude,x.longitude),axis=1)
winddirection_raw["region"]=winddirection_raw.apply(lambda x:get_region(x.latitude,x.longitude),axis=1)
windspeed_raw["region"]=windspeed_raw.apply(lambda x:get_region(x.latitude,x.longitude),axis=1)
rainfall_raw["region"]=rainfall_raw.apply(lambda x:get_region(x.latitude,x.longitude),axis=1)

Once again, this step could potentially take a long time and so the next step is to save any progress thus far and is optional.

In [ ]:
#temperature_raw.to_csv("temperature_transformed.csv", index=False)
#humidity_raw.to_csv("humidity_transformed.csv", index=False)
#winddirection_raw.to_csv("winddirection_transformed.csv", index=False)
#windspeed_raw.to_csv("windspeed_transformed.csv", index=False)
#rainfall_raw.to_csv("rainfall_transformed.csv", index=False)

Aggregating by timestamp and region, we realize that for rainfall, we can simply take the mean and then convert it into a binary afterwards.

In [31]:
temperature=temperature_raw.groupby(["timestamp","region"])["reading"].mean().to_frame().reset_index()
humidity=humidity_raw.groupby(["timestamp","region"])["reading"].mean().to_frame().reset_index()
winddirection=winddirection_raw.groupby(["timestamp","region"])["reading"].mean().to_frame().reset_index()
windspeed=windspeed_raw.groupby(["timestamp","region"])["reading"].mean().to_frame().reset_index()
rainfall=rainfall_raw.groupby(["timestamp","region"])["reading"].mean().to_frame().reset_index()

In [33]:
rainfall["reading"]=rainfall["reading"].apply(lambda x: 1 if x>0 else 0)
rainfall

,timestamp,region,reading
0,2017-01-01 00:00:00,central,0
1,2017-01-01 00:00:00,east,0
2,2017-01-01 00:00:00,north,0
3,2017-01-01 00:00:00,north-east,0
4,2017-01-01 00:00:00,west,0
...,...,...,...
216166,2021-12-31 23:00:00,central,1
216167,2021-12-31 23:00:00,east,0
216168,2021-12-31 23:00:00,north,0
216169,2021-12-31 23:00:00,north-east,0


In [49]:
weather=temperature.merge(humidity, on=["timestamp","region"],suffixes=('_temperature', '_humidity'))
weather=weather.merge(winddirection, on=["timestamp","region"])
weather=weather.merge(windspeed, on=["timestamp","region"],suffixes=('_winddirection', '_windspeed'))
weather=weather.merge(rainfall, on=["timestamp","region"]).rename(columns={"reading":"rainfall"}).rename(columns=lambda x: x.replace("reading_",""))
weather

,timestamp,region,temperature,humidity,winddirection,windspeed,rainfall
0,2017-01-01 00:00:00,central,27.000000,89.350000,246.333333,7.166667,0
1,2017-01-01 00:00:00,east,26.275000,87.333333,260.750000,1.650000,0
2,2017-01-01 00:00:00,north,25.866667,85.500000,119.666667,1.833333,0
3,2017-01-01 00:00:00,north-east,26.000000,89.300000,26.333333,2.366667,0
4,2017-01-01 00:00:00,west,26.480000,84.333333,211.600000,4.400000,0
...,...,...,...,...,...,...,...
210160,2021-12-31 23:00:00,central,24.233333,96.950000,109.500000,3.225000,1
210161,2021-12-31 23:00:00,east,24.600000,92.200000,238.000000,4.900000,0
210162,2021-12-31 23:00:00,north,24.900000,95.100000,94.000000,4.000000,0
210163,2021-12-31 23:00:00,north-east,24.650000,91.450000,145.000000,3.100000,0


### Adjustment 2: Decomposing wind into windspeed in 2 directions

Similar to the reason we gave for planning areas being not ideal, we believe that having too many categories of wind directions (16-point compass) could have affected the results. At the same time, further research showed other ways of processing the wind data including decomposing wind speeds into the Eastings (X) and Northings (Y) directions which would make more intuitive sense for cases where windspeeds are zero. i.e. When wind speed is zero, it is meaningless to have a wind direction.

<img src="Images/trigonometry.png" width=400 align="left">

Hence, using the trigonometric functions, sine and cosine, we can convert the wind speed values into a wind speed in the X and Y direction.

In [57]:
weather["wind_x"]=0
weather["wind_y"]=0
for i in range(len(weather)):
    weather["wind_x"][i]=math.sin(math.radians(weather["winddirection"][i]))*weather["windspeed"][i]
    weather["wind_y"][i]=math.cos(math.radians(weather["winddirection"][i]))*weather["windspeed"][i]
weather

<ipython-input-57-63f62081d444>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather["wind_x"][i]=math.sin(math.radians(weather["winddirection"][i]))*weather["windspeed"][i]
<ipython-input-57-63f62081d444>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather["wind_y"][i]=math.cos(math.radians(weather["winddirection"][i]))*weather["windspeed"][i]


,timestamp,region,temperature,humidity,winddirection,windspeed,rainfall,wind_x,wind_y
0,2017-01-01 00:00:00,central,27.000000,89.350000,246.333333,7.166667,0,-6,-2
1,2017-01-01 00:00:00,east,26.275000,87.333333,260.750000,1.650000,0,-1,0
2,2017-01-01 00:00:00,north,25.866667,85.500000,119.666667,1.833333,0,1,0
3,2017-01-01 00:00:00,north-east,26.000000,89.300000,26.333333,2.366667,0,1,2
4,2017-01-01 00:00:00,west,26.480000,84.333333,211.600000,4.400000,0,-2,-3
...,...,...,...,...,...,...,...,...,...
210160,2021-12-31 23:00:00,central,24.233333,96.950000,109.500000,3.225000,1,3,-1
210161,2021-12-31 23:00:00,east,24.600000,92.200000,238.000000,4.900000,0,-4,-2
210162,2021-12-31 23:00:00,north,24.900000,95.100000,94.000000,4.000000,0,3,0
210163,2021-12-31 23:00:00,north-east,24.650000,91.450000,145.000000,3.100000,0,1,-2


### Adjustment 3: Calculate the Changes in numerical values from the previous period

Lastly, we believe that the Change in weather variables as opposed to the absolute readings of the weather vairables themselves may help to improve the results.

In [59]:
runtimes=list(pd.date_range('2017-01-01 00:00:00',
                            '2021-12-31 23:59:59',
                            freq='60T').strftime('%Y-%m-%d %H:%M:%S'))
available_areas=list(weather["region"].unique())
expected_rows=len(runtimes)*len(available_areas)
expected_rows

219120

While there were 0 missing rows introduced during the scraping, the dataset actually lacks data for EVERY single timestamp for EVERY single region. Hence, we intoriduce back some rows of NA to make the dataset not have any consecutive period gaps by using a left join.

In [60]:
expanded_runtime=pd.DataFrame(runtimes,columns=["timestamp"])
expanded_runtime['key'] = 1

expanded_areas=pd.DataFrame(available_areas,columns=["region"])
expanded_areas['key'] = 1

expanded_df=expanded_runtime.merge(expanded_areas, how="outer",on="key").drop("key", 1)
expanded_df

,timestamp,region
0,2017-01-01 00:00:00,central
1,2017-01-01 00:00:00,east
2,2017-01-01 00:00:00,north
3,2017-01-01 00:00:00,north-east
4,2017-01-01 00:00:00,west
...,...,...
219115,2021-12-31 23:00:00,central
219116,2021-12-31 23:00:00,east
219117,2021-12-31 23:00:00,north
219118,2021-12-31 23:00:00,north-east


In [74]:
weather_full=expanded_df.merge(weather,on=["timestamp","region"],how="left")
weather_full=weather_full.drop(columns=["windspeed","winddirection"])
weather_full

,timestamp,region,temperature,humidity,rainfall,wind_x,wind_y
0,2017-01-01 00:00:00,central,27.000000,89.350000,0.0,-6.0,-2.0
1,2017-01-01 00:00:00,east,26.275000,87.333333,0.0,-1.0,0.0
2,2017-01-01 00:00:00,north,25.866667,85.500000,0.0,1.0,0.0
3,2017-01-01 00:00:00,north-east,26.000000,89.300000,0.0,1.0,2.0
4,2017-01-01 00:00:00,west,26.480000,84.333333,0.0,-2.0,-3.0
...,...,...,...,...,...,...,...
219115,2021-12-31 23:00:00,central,24.233333,96.950000,1.0,3.0,-1.0
219116,2021-12-31 23:00:00,east,24.600000,92.200000,0.0,-4.0,-2.0
219117,2021-12-31 23:00:00,north,24.900000,95.100000,0.0,3.0,0.0
219118,2021-12-31 23:00:00,north-east,24.650000,91.450000,0.0,1.0,-2.0


<code>.diff</code> and <code>.shift()</code> functions are used to create past and delta weather variables and upon merging with the main dataset, we will drop the NA values.

In [70]:
weather_shifted=weather_full.iloc[:,1:].groupby(["region"]).shift(1)
weather_shifted=weather_shifted.add_prefix("past_")
weather_shifted

,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
219115,23.666667,95.400000,1.0,-1.0,-1.0
219116,24.800000,93.000000,1.0,-3.0,0.0
219117,24.500000,96.200000,1.0,0.0,-1.0
219118,24.300000,92.650000,0.0,0.0,-1.0


In [69]:
weather_diff=weather_full.iloc[:,1:].groupby(["region"]).diff(1)
weather_diff=weather_diff.add_prefix("delta_")
weather_diff

,delta_temperature,delta_humidity,delta_rainfall,delta_wind_x,delta_wind_y
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
219115,0.566667,1.55,0.0,4.0,0.0
219116,-0.200000,-0.80,-1.0,-1.0,-2.0
219117,0.400000,-1.10,-1.0,3.0,1.0
219118,0.350000,-1.20,0.0,1.0,-1.0


In [76]:
weather_full=pd.concat([weather_full,weather_shifted,weather_diff],axis=1)
weather_full=weather_full.dropna().reset_index(drop=True)
weather_full=pd.concat([weather_full.iloc[:,:2],weather_full.iloc[:,7:],weather_full.iloc[:,4:5]],axis=1)
weather_full

,timestamp,region,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_rainfall,...,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_rainfall,delta_wind_x,delta_wind_y,rainfall
0,2017-01-01 02:00:00,central,26.950000,90.300000,0.0,-6.0,-4.0,-0.100000,-1.421085e-14,0.0,...,89.350000,0.0,-6.0,-2.0,-0.050000,0.950000,0.0,0.0,-2.0,0.0
1,2017-01-01 02:00:00,east,26.275000,87.966667,0.0,-1.0,0.0,-0.150000,-5.333333e-01,0.0,...,87.333333,0.0,-1.0,0.0,0.000000,0.633333,0.0,0.0,0.0,0.0
2,2017-01-01 02:00:00,north,26.100000,86.600000,0.0,-1.0,2.0,-0.100000,4.000000e-01,0.0,...,85.500000,0.0,1.0,0.0,0.233333,1.100000,0.0,-2.0,2.0,0.0
3,2017-01-01 02:00:00,north-east,25.966667,89.400000,0.0,1.0,2.0,0.033333,-1.500000e-01,0.0,...,89.300000,0.0,1.0,2.0,-0.033333,0.100000,0.0,0.0,0.0,0.0
4,2017-01-01 02:00:00,west,26.240000,86.200000,0.0,-4.0,0.0,-0.140000,8.666667e-01,0.0,...,84.333333,0.0,-2.0,-3.0,-0.240000,1.866667,0.0,-2.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197640,2021-12-31 23:00:00,central,23.666667,95.400000,1.0,-1.0,-1.0,0.566667,1.550000e+00,0.0,...,90.866667,0.0,1.0,-2.0,0.033333,-0.166667,0.0,1.0,1.0,1.0
197641,2021-12-31 23:00:00,east,24.800000,93.000000,1.0,-3.0,0.0,-0.200000,-8.000000e-01,-1.0,...,93.450000,0.0,0.0,-1.0,0.166667,-2.150000,0.0,0.0,2.0,0.0
197642,2021-12-31 23:00:00,north,24.500000,96.200000,1.0,0.0,-1.0,0.400000,-1.100000e+00,-1.0,...,89.100000,0.0,0.0,1.0,0.000000,1.200000,0.0,-1.0,0.0,0.0
197643,2021-12-31 23:00:00,north-east,24.300000,92.650000,0.0,0.0,-1.0,0.350000,-1.200000e+00,0.0,...,93.600000,0.0,0.0,-4.0,-0.200000,-0.800000,0.0,0.0,0.0,0.0


However, the current delta variables represent the CHANGE in weather variables now. Since the analysis plan is to use only past data to predict. We still need to shift this by 1 period to represent the CHANGE in weather variables experienced 1 HOUR AGO.

To illustrate:

<img src="Images/REadjustment.png" align="left">

In [5]:
weather_full=pd.concat([weather_full.iloc[:,:9],weather_full.iloc[:,10:12],weather_full.iloc[:,22:]],axis=1)
weather_full

,timestamp,region,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y,rainfall
0,2017-01-01 02:00:00,central,26.950000,90.300000,0.0,-6.0,-4.0,-0.100000,-1.421085e-14,0.0,0.0,0.0
1,2017-01-01 02:00:00,east,26.275000,87.966667,0.0,-1.0,0.0,-0.150000,-5.333333e-01,0.0,1.0,0.0
2,2017-01-01 02:00:00,north,26.100000,86.600000,0.0,-1.0,2.0,-0.100000,4.000000e-01,0.0,0.0,0.0
3,2017-01-01 02:00:00,north-east,25.966667,89.400000,0.0,1.0,2.0,0.033333,-1.500000e-01,-2.0,-1.0,0.0
4,2017-01-01 02:00:00,west,26.240000,86.200000,0.0,-4.0,0.0,-0.140000,8.666667e-01,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
197640,2021-12-31 23:00:00,central,23.666667,95.400000,1.0,-1.0,-1.0,0.566667,1.550000e+00,4.0,0.0,1.0
197641,2021-12-31 23:00:00,east,24.800000,93.000000,1.0,-3.0,0.0,-0.200000,-8.000000e-01,-1.0,-2.0,0.0
197642,2021-12-31 23:00:00,north,24.500000,96.200000,1.0,0.0,-1.0,0.400000,-1.100000e+00,3.0,1.0,0.0
197643,2021-12-31 23:00:00,north-east,24.300000,92.650000,0.0,0.0,-1.0,0.350000,-1.200000e+00,1.0,-1.0,0.0


In [13]:
weather_past_delta=weather_full.groupby("region").shift(1)
weather_past_delta=weather_past_delta.iloc[:,6:10]
weather_past_delta

,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
197640,-0.766667,-2.150000,-3.0,1.0
197641,0.100000,-0.300000,-2.0,-2.0
197642,0.200000,-0.600000,-2.0,-1.0
197643,0.350000,-1.000000,0.0,-3.0


In [15]:
weather_full=pd.concat([weather_full.iloc[:,:7],weather_past_delta,weather_full.iloc[:,11:]],axis=1)
weather_full=weather_full.dropna().reset_index(drop=True)
weather_full

,timestamp,region,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y,rainfall
0,2017-01-01 03:00:00,central,26.850000,90.300000,0.0,-6.0,-4.0,-0.100000,-1.421085e-14,0.0,0.0,0.0
1,2017-01-01 03:00:00,east,26.125000,87.433333,0.0,-1.0,1.0,-0.150000,-5.333333e-01,0.0,1.0,0.0
2,2017-01-01 03:00:00,north,26.000000,87.000000,0.0,-1.0,2.0,-0.100000,4.000000e-01,0.0,0.0,0.0
3,2017-01-01 03:00:00,north-east,26.000000,89.250000,0.0,-1.0,1.0,0.033333,-1.500000e-01,-2.0,-1.0,0.0
4,2017-01-01 03:00:00,west,26.100000,87.066667,0.0,-4.0,0.0,-0.140000,8.666667e-01,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
197635,2021-12-31 23:00:00,central,23.666667,95.400000,1.0,-1.0,-1.0,-0.766667,-2.150000e+00,-3.0,1.0,1.0
197636,2021-12-31 23:00:00,east,24.800000,93.000000,1.0,-3.0,0.0,0.100000,-3.000000e-01,-2.0,-2.0,0.0
197637,2021-12-31 23:00:00,north,24.500000,96.200000,1.0,0.0,-1.0,0.200000,-6.000000e-01,-2.0,-1.0,0.0
197638,2021-12-31 23:00:00,north-east,24.300000,92.650000,0.0,0.0,-1.0,0.350000,-1.000000e+00,0.0,-3.0,0.0


In [16]:
from collections import Counter
Counter(weather_full.rainfall)

Counter({0.0: 181166, 1.0: 16474})

In [17]:
weather_full.to_csv("weather_data_2.csv",index=False)